## Description

Smith-Waterman algorithm in O3 time.

Part of the code has been blanked out with X's. Fill in this code to make the code run 

## Python Imports

In [1]:
import numpy as np

## Data Imports

## DEFINE THE PATH TO YOUR COURSE DIRECTORY

In [2]:
data_dir = "/home/mathias/bioinfo_algos/data/"

## Define runtime parameters

In [3]:
gap_open = -11
gap_extension = -1

### Alphabet

In [4]:
#alphabet_file = "https://raw.githubusercontent.com/brunoalvarez89/data/master/algorithms_in_bioinformatics/part_3/alphabet"
alphabet_file = data_dir + "Matrices/alphabet"
alphabet = np.loadtxt(alphabet_file, dtype=str)

alphabet

array(['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M',
       'F', 'P', 'S', 'T', 'W', 'Y', 'V'], dtype='<U1')

### Blosum Matrix

In [5]:
#blosum_file = "https://raw.githubusercontent.com/brunoalvarez89/data/master/algorithms_in_bioinformatics/part_3/blosum50"
blosum_file = data_dir + "Matrices/BLOSUM50"

_blosum50 = np.loadtxt(blosum_file, dtype=float).reshape((24, -1)).T
blosum50 = {}

for i, letter_1 in enumerate(alphabet):
    
    blosum50[letter_1] = {}

    for j, letter_2 in enumerate(alphabet):
        
        blosum50[letter_1][letter_2] = _blosum50[i, j]

blosum50

{'A': {'A': 5.0,
  'R': -2.0,
  'N': -1.0,
  'D': -2.0,
  'C': -1.0,
  'Q': -1.0,
  'E': -1.0,
  'G': 0.0,
  'H': -2.0,
  'I': -1.0,
  'L': -2.0,
  'K': -1.0,
  'M': -1.0,
  'F': -3.0,
  'P': -1.0,
  'S': 1.0,
  'T': 0.0,
  'W': -3.0,
  'Y': -2.0,
  'V': 0.0},
 'R': {'A': -2.0,
  'R': 7.0,
  'N': -1.0,
  'D': -2.0,
  'C': -4.0,
  'Q': 1.0,
  'E': 0.0,
  'G': -3.0,
  'H': 0.0,
  'I': -4.0,
  'L': -3.0,
  'K': 3.0,
  'M': -2.0,
  'F': -3.0,
  'P': -3.0,
  'S': -1.0,
  'T': -1.0,
  'W': -3.0,
  'Y': -1.0,
  'V': -3.0},
 'N': {'A': -1.0,
  'R': -1.0,
  'N': 7.0,
  'D': 2.0,
  'C': -2.0,
  'Q': 0.0,
  'E': 0.0,
  'G': 0.0,
  'H': 1.0,
  'I': -3.0,
  'L': -4.0,
  'K': 0.0,
  'M': -2.0,
  'F': -4.0,
  'P': -2.0,
  'S': 1.0,
  'T': 0.0,
  'W': -4.0,
  'Y': -2.0,
  'V': -3.0},
 'D': {'A': -2.0,
  'R': -2.0,
  'N': 2.0,
  'D': 8.0,
  'C': -4.0,
  'Q': 0.0,
  'E': 2.0,
  'G': -1.0,
  'H': -1.0,
  'I': -4.0,
  'L': -4.0,
  'K': -1.0,
  'M': -4.0,
  'F': -5.0,
  'P': -1.0,
  'S': 0.0,
  'T': -1.0,


## Alignment

This functions returns, apart from the final Alignment Matrix, all the intermedite Matrices (for plotting purposes).

### Alignment Matrix

In [10]:
def smith_waterman_alignment(query="VLLP", database="VLILP", scoring_scheme={}, gap_open=-5, gap_extension=-1):
    
    # Matrix dimensions
    M = len(query)   # M = 4
    N = len(database) # N = 5
    
    # E matrix (for backtracking)
    E_matrix = np.zeros((M+1, N+1), dtype=object)
    
    # D matrix (alignment matrix)
    D_matrix = np.zeros((M+1, N+1), int)

    # Initialize matrices (Here you might add values to penalize end gaps)
    for i in range(M, 0, -1):
        D_matrix[i-1, N] = 0
        E_matrix[i-1, N] = 0

    for j in range(N, 0, -1):
        D_matrix[M, j-1] = 0
        E_matrix[M, j-1] = 0
    
    
    D_matrix_max_score, D_matrix_i_max, D_matrix_j_max = -9, -9, -9
    for i in range(M-1, -1, -1):         # i goes from 3 to 0 
        for j in range(N-1, -1, -1):     # j goes from 4 to 0 
                
            # digonal score
            diagonal_score = D_matrix[i+1,j+1] + scoring_scheme[query[i]][database[j]]
            # horizontal score
            # Gap opening
            max_horizontal_score = D_matrix[i, j+1] + gap_open
            k_max_horizontal_score = 1
          
            # Gap extensions
            for k in range(j+2, N): # k goes from (6, 5, 4, 3, 2) to 5
                
                score = D_matrix[i, k] + gap_open + (k-j-1)*gap_extension
                
                if score > max_horizontal_score: 
                    max_horizontal_score = score 
                    k_max_horizontal_score = k - j            
            
            
            # vertical score
            # Gap opening
            max_vertical_score = D_matrix[i+1, j] + gap_open
            k_max_vertical_score = 1
            
            # Gap extensions
            for k in range(i+2, M):
    
                score =  D_matrix[k, j] + gap_open + (k-i-1)*gap_extension
               
                if score > max_vertical_score: 
                    max_vertical_score = score 
                    k_max_vertical_score = k - i
                  
                  
            ####################
            # E_matrix entries #
            ####################
            # E[i,j] = 0, negative number
            # E[i,j] = 1, match
            # E[i,j] = 2, gap opening in database
            # E[i,j] = 3, gap extension in database
            # E[i,j] = 4, gap opening in query
            # E[i,j] = 5, gap extension in query
            
            if diagonal_score >= max_vertical_score and diagonal_score >= max_horizontal_score:
                max_score = diagonal_score
                direction = "diagonal"
            elif max_horizontal_score > max_vertical_score:
                max_score = max_horizontal_score
                direction = "horizontal"
            else:
                max_score = max_vertical_score
                direction = "vertical"
                
            if max_score <= 0:
                max_score = 0
                direction = "none"

            # diagonal direction case
            if direction == "diagonal":
                E_matrix[i,j] = 1
                
            # vertical direction case
            elif direction == "vertical":

                # if k only moved one position, it means gap opening
                if k_max_vertical_score == 1: 
                    E_matrix[i,j] = 2

                # else it is a gap extension
                else: 
                    E_matrix[i,j] = 3
                        
            # horizontal direction case
            elif direction == "horizontal":

                # if k only moved one position, it means gap opening
                if k_max_horizontal_score == 1: 
                    E_matrix[i,j] = 4

                # else it is a gap extension
                else: 
                    E_matrix[i,j] = 5

            else:
                # max_score is negative, put E to zero
                E_matrix[i,j] = 0
                 
            # store max score
            D_matrix[i, j] = max_score
            
            # append partial alignment matrix to list
            #D_matrix_list.append(np.copy(D_matrix))
           
            # fetch global max score
            if max_score > D_matrix_max_score:
                D_matrix_max_score = max_score
                D_matrix_i_max = i
                D_matrix_j_max = j
            
    return D_matrix, E_matrix, D_matrix_i_max, D_matrix_j_max, D_matrix_max_score

## Alignment Matrix Traceback

In [11]:
def smith_waterman_traceback(E_matrix, D_matrix, i_max, j_max, query="VLLP", database="VLILP", gap_open=-5, gap_extension=-1):
    
    M = len(query)
    N = len(database)
    
    aligned_query = []
    aligned_database = []
    positions = []
    matches = 0
    
    # start from max_i, max_j
    i, j = i_max, j_max
    while i < M and j < N :

        positions.append([i,j])
        
        # E[i,j] = 0, stop back tracking
        if E_matrix[i, j] == 0:
            break
        
        # E[i,j] = 1, match
        if E_matrix[i, j] == 1:
            aligned_query.append(query[i])
            aligned_database.append(database[j])
            if (query[i] == database[j]):
                matches += 1
            i += 1
            j += 1
        
        
        # E[i,j] = 2, gap opening in database
        if E_matrix[i, j] == 2:
            aligned_database.append("-")
            aligned_query.append(query[i])
            i += 1

            
        # E[i,j] = 3, gap extension in database
        if E_matrix[i, j] == 3:
            
            count = i + 2
            score = D_matrix[count, j] + gap_open + gap_extension

            # Find length of gap (check if score == D_matrix[i, j])
            while((score - D_matrix[i, j])*(score - D_matrix[i, j]) >= 0.00001): 
                count += 1
                score = D_matrix[count, j] + gap_open + (count-i-1)*gap_extension

            for k in range(i, count):
                aligned_database.append("-")
                aligned_query.append(query[i])
                i += 1
             
          
        # E[i,j] = 4, gap opening in query
        if E_matrix[i, j] == 4:
            aligned_query.append("-")
            aligned_database.append(database[j])
            j += 1
        
        
        # E[i,j] = 5, gap extension in query
        if E_matrix[i, j] == 5:
            
            count = j + 2
            score = D_matrix[i, count] + gap_open + gap_extension
            
            # Find length of gap (check if score == D_matrix[i, j])
            while((score - D_matrix[i, j])*(score - D_matrix[i, j]) >= 0.0001): 
                count += 1
                score = D_matrix[i, count] + gap_open + (count-j-1)*gap_extension

            for k in range(j, count):
                aligned_query.append("-")
                aligned_database.append(database[j])
                j += 1
                

    return aligned_query, aligned_database, matches

## Now test the code on a few examples

In [16]:
#Slides example
#query = "VLLP"
#database = "VLILP"
#scoring_scheme = blosum50
#gap_open = -5
#gap_extension = -1

#Matrix dump exercise 2
#query = "VLPVLILP"
#database = "VLLPVLLP"
#scoring_scheme = blosum50
#gap_open = -2
#gap_extension = -1

#Matrix dump exercise 1
query = "IDVLLGADDGSLAFVPSEFSISPGEKIVFKNNAGFPHNIVFDEDSIPSGVDASKISMSEEDLLNAKGETFEVALSNKGEYSFYCSPHQGAGMVGKVTVN"
database = "AEVKLGSDDGGLVFSPSSFTVAAGEKITFKNNAGFPHNIVFDEDEVPAGVNAEKISQPEYLNGAGETYEVTLTEKGTYKFYCEPHAGAGMKGEVTVN"
scoring_scheme = blosum50
gap_open = -11
gap_extension = -1

D_matrix, E_matrix, i_max, j_max, max_score = smith_waterman_alignment(query, database, scoring_scheme, gap_open, gap_extension)
aligned_query, aligned_database, matches = smith_waterman_traceback(E_matrix, D_matrix, i_max, j_max, query, database, gap_open, gap_extension)

print("ALN", query, len(query), database, len(database), len(aligned_query), matches, max_score)
print("QAL", ''.join(aligned_query))
print("DAL", ''.join(aligned_database))
print("")

print("---")

print("D Matrix")
print(D_matrix)
print("")
print("E Matrix")
print(E_matrix)

print("---")

ALN IDVLLGADDGSLAFVPSEFSISPGEKIVFKNNAGFPHNIVFDEDSIPSGVDASKISMSEEDLLNAKGETFEVALSNKGEYSFYCSPHQGAGMVGKVTVN 99 AEVKLGSDDGGLVFSPSSFTVAAGEKITFKNNAGFPHNIVFDEDEVPAGVNAEKISQPEYLNGAGETYEVTLTEKGTYKFYCEPHAGAGMKGEVTVN 97 98 64 439.0
QAL DVLLGADDGSLAFVPSEFSISPGEKIVFKNNAGFPHNIVFDEDSIPSGVDASKISMSEEDLLNAKGETFEVALSNKGEYSFYCSPHQGAGMVGKVTVN
DAL EVKLGSDDGGLVFSPSSFTVAAGEKITFKNNAGFPHNIVFDEDEVPAGVNAEKISQPE--YLNGAGETYEVTLTEKGTYKFYCEPHAGAGMKGEVTVN

---
D Matrix
[[438 428 426 ...   6   0   0]
 [428 439 426 ...   0   2   0]
 [425 426 437 ...   5   0   0]
 ...
 [  0   0   5 ...  12   0   0]
 [  0   0   0 ...   0   7   0]
 [  0   0   0 ...   0   0   0]]

E Matrix
[[1 2 1 ... 1 0 0]
 [4 1 2 ... 0 1 0]
 [5 4 1 ... 1 0 0]
 ...
 [0 0 1 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
---


## Multiple alignments. Run an alignment of a sequence against a sequence database


### Load 1PLC.tab (query)


In [13]:
#query_file = "https://raw.githubusercontent.com/brunoalvarez89/data/master/algorithms_in_bioinformatics/part_3/1PLC._.tab"
query_file = data_dir + "Align/1PLC._.tab"
query_list = np.loadtxt(query_file, dtype=str).reshape(-1,2)

### Load database_list.tab

In [14]:
#database_file = "https://raw.githubusercontent.com/brunoalvarez89/data/master/algorithms_in_bioinformatics/part_3/database_list.tab"
database_file = data_dir + "Align/db_100.tab"
database_list = np.loadtxt(database_file, dtype=str).reshape(-1,2)

### Align query against database. Might take a while. Go get some coffee 

In [15]:
from time import time

scoring_scheme = blosum50
gap_open = -11
gap_extension = -1

# this returns current timestamp in seconds
t0 = time()

for query in query_list:
    
    query_protein = query[0]
    query_sequence = query[1]
    
    for database in database_list:
    
        database_protein = database[0]
        database_sequence = database[1]
    
        D_matrix, E_matrix, i_max, j_max, max_score = smith_waterman_alignment(query_sequence, database_sequence, scoring_scheme, gap_open, gap_extension)
        aligned_query, aligned_database, matches = smith_waterman_traceback(E_matrix, D_matrix, i_max, j_max, query_sequence, database_sequence, gap_open, gap_extension)
        
        print("ALN", query_protein, len(query_sequence), database_protein, len(database_sequence), len(aligned_query), matches, max_score)
        print("QAL", i_max, ''.join(aligned_query))
        print("DAL", j_max,''.join(aligned_database))
        print("")
        
t1 = time()

print( "Time (m):", (t1-t0)/60)

ALN 1PLC._ 99 1US0.A 316 115 29 59.0
QAL 1 DVLLGADDGSLAFVPSEFSISPGEKIVFKNNAG--FPHNI-VFD-----EDSIPSG-VDASKIS----MSEEDLLNAKGETFEVALSNKGEYSFYCSPH----------QGAGMV
DAL 98 DLKLDYLDLYLIHWPTGFK--PGKEFFPLDESGNVVPSDTNILDTWAAMEELVDEGLVKAIGISNFNHLQVEMILNKPGLKYKPAV-NQIE----CHPYLTQEKLIQYCQSKGIV

ALN 1PLC._ 99 2EAB.A 899 104 24 70.0
QAL 1 DVLL--GADDGSLAFVPSEFSISPGEK-IVFKNN---------AGFPHNIV-------------FDEDSIPSGVDASKISMSEEDLLNAKGETFEV--ALSNKG
DAL 140 DIYLDYGFNDTTVTEYRRDLNLSKGKADVTFKHDGVTYTREYFASNPDNVMVARLTASKAGKLNFNV-SMPTNTNYSKTG----ETTTVKGDTLTVKGALGNNG

ALN 1PLC._ 99 1QWN.A 1045 59 15 54.0
QAL 5 GADDGSLAFVPSEFSISPGEKIVFKNNAGFPHNIVFDEDSIPSGVDASKISMSEEDLLN
DAL 534 GVEDSRTTIILGE-DILPSKHVVMHNT--LPH---WREQLVDFYVSSPFVSVT--DLAN

ALN 1PLC._ 99 1HLR.A 907 73 23 61.0
QAL 5 GADDGSL-----AFVPSEFSISPGEKIVFK--NNAGFPHNIVFDEDSIPSG------VDASKISMSEEDLLNA
DAL 655 GADIGCVGTAHEALRP--MGVAP-EKIKFTWPNTATTP-------NSGPSGGSREQVMTGNAIRVACENLLKA

ALN 1PLC._ 99 1MXT.A 504 116 28 48.0
QAL 4 LGAD--DGSLAFVPSEFSISP-GEKI-------VFKN--NA

ALN 1PLC._ 99 1WDP.A 495 103 26 55.0
QAL 5 GADDGSLAFVP-SEFSISPGEK---IVFKNNAGFPHN----IVFDEDSIPSGVDASKI------SMSEE--DLLNAKGETF--EVALSNKGEYSFYCSPH-QG
DAL 95 GGNVGDIVNIPIPQWVLDIGESNHDIFYTNRSGTRNKEYLTVGVDNEPIFHGRTAIEIYSDYMKSFRENMSDFLES-GLIIDIEVGLGPAGELRYPSYPQSQG

ALN 1PLC._ 99 1HDH.A 536 93 21 51.0
QAL 18 FSISPGEKIVFKNNAGF--PHN-------------IVFDE---DSIPSGVDASKISMSEEDLLNAKGETFEVALSNKGE------YSFYCSPH
DAL 132 FSLLPGAA----NHYGFEPPYDESTPRILKGTPALYVEDERYLDTLPEGFYSS----------DAFGDKLLQYLKERDQSRPFFAYLPFSAPH

ALN 1PLC._ 99 1WVF.A 520 33 13 47.0
QAL 27 VFKNNAGFPHNIVF-DEDSIPSGVDASK-ISMS
DAL 242 VFK-----PFEVIFEDEADIVEIVDALRPLRMS

ALN 1PLC._ 99 2BMO.A 447 94 26 66.0
QAL 23 GEKIVFKNNAG--FPHNIVF-DEDS-IPS---------GVDASKISMSEED----LLNA---KGETFEVALSNKGEYSFYCSPH-QGAGMVGKV
DAL 19 GDKELFQHELKTIFARNWLFLTHDSLIPSPGDYVKAKMGVDEVIVSRQNDGSVRAFLNVCRHRGKTLVHAEAGNAK-GFVCGYHGWGYGSNGEL

ALN 1PLC._ 99 2J6L.B 500 49 12 48.0
QAL 4 LGADDGSLAFVPSEFS-ISPGEKIVFKNNAG---------FPHNIVFDE
DAL 269 LGGNNAIIAFEDADLSLVVPSALFAAVGTAGQRCTTARRL

ALN 1PLC._ 99 1Z53.A 294 84 19 43.0
QAL 23 GEKIVFK-NNAGFPHNIVFDEDSIPSG-VDAS-------KISMSEEDLL-----NAKGETFEVALSNKGEYSFYCSPHQGAGMV
DAL 120 GPKIPWRCGRVDTPEDTTPDNGRLPDADKDADYVRTFFQRLNMNDREVVALMGAHALGKTH---LKNSG----YEGPWGAANNV

ALN 1PLC._ 99 1MOO.A 260 51 16 46.0
QAL 0 IDVLLGADDGSLAFVPSEFSISPGEKI-VFKNNAGFPHNIVFDEDSIPSGV
DAL 32 IDTHTAKYDPSLK--PLSVSYDQATSLRILNNGAAF--NVEFD-DSQDKAV

ALN 1PLC._ 99 1ZJY.A 251 67 17 52.0
QAL 7 DDGSLAFVPSEFSISPGEKIVFKNNAGFPHNIVF------DEDSIPSGVDASKISMSE-EDLLNAKG
DAL 27 EEGAKVMITDRHS-DVGEKAA--KSVGTPDQIQFFQHDSSDEDGWTKLFDATEKAFGPVSTLVNNAG

ALN 1PLC._ 99 1QSA.A 618 35 11 50.0
QAL 13 FVPSEFSISPGE-KIVFKNNAGFPHNIVFDEDSIP
DAL 546 FSSAAYNAGPGRVRTWLGNSAGRIDAVAFVE-SIP

ALN 1PLC._ 99 2CIW.A 299 67 17 56.0
QAL 12 AFVPSEFSISP--GEKIVFKNNAGFPHNIVFDEDSIPSGVDASKISMSEEDLLNAKG---ETFEVAL
DAL 75 AFVVCEYVTGSDCGDSLVNLTLLAEPH--AFEHDHSFSRKDYKQGVANSNDFIDNRNFDAETFQTSL

ALN 1PLC._ 99 1EDQ.A 540 86 20 54.0
QAL 21 SPGEKIVFKNNAGFPHNIVFDEDSIPSGVD-------ASKISMSEEDLLNAK-GETFEVALSNKGEYSFYCSPHQGAGMVGKVT